In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [2]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from skforecast.metrics import mean_absolute_scaled_error

from skforecast.metrics import add_y_train_argument
from skforecast.model_selection.model_selection import _get_metric

In [3]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

# Train-validation dates
# ==============================================================================
end_train = '2002-01-01 23:59:00'

print(
    f"Train dates      : {data.index.min()} --- {data.loc[:end_train].index.max()}"
    f"  (n={len(data.loc[:end_train])})"
)
print(
    f"Validation dates : {data.loc[end_train:].index.min()} --- {data.index.max()}"
    f"  (n={len(data.loc[end_train:])})"
)


# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = False,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True  
                      )

print(f"Backtesting MASE: {metric}")

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)
Train dates      : 1991-07-01 00:00:00 --- 2002-01-01 00:00:00  (n=127)
Validation dates : 2002-02-01 00:00:00 --- 2008-06-01 00:00:00  (n=77)


  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting MASE: 2.344471182248395


In [4]:
mae_foreast = mean_absolute_error(data.loc[end_train:], predictions)
naive_in_sample_foreast = data.loc[:end_train].shift(1).dropna()
mae_in_sample = mean_absolute_error(data.loc[naive_in_sample_foreast.index], naive_in_sample_foreast)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")

mae forecast: 0.19922359134545478
mae in sample: 0.08497591817460318
mase: 2.344471182248395


In [5]:
y_true = data.loc[end_train:]
y_pred = predictions['pred']
y_train = data.loc[:end_train]
mean_absolute_scaled_error(y_true, y_pred, y_train)

2.344471182248395

In [6]:
metric = _get_metric("mean_absolute_error")
metric(y_true=y_true, y_pred=y_pred, y_train=y_train)

0.19922359134545478

In [7]:
metric = add_y_train_argument(mean_absolute_error)
metric(y_true=y_true, y_pred=y_pred, y_train=y_train)

0.19922359134545478